<a href="https://colab.research.google.com/github/Vishnuanand77/DataAnalyticsProject/blob/master/F1_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor

np.set_printoptions(precision=4)

In [8]:
data = pd.read_csv('final_df.csv')
data.head()

,season,round,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy,driver,grid,podium,driver_points,driver_wins,driver_standings_pos,constructor_points,constructor_wins,constructor_standings_pos,qualifying_time,driver_age,circuit_id_BAK,circuit_id_adelaide,circuit_id_albert_park,circuit_id_americas,circuit_id_bahrain,circuit_id_brands_hatch,circuit_id_catalunya,circuit_id_detroit,circuit_id_estoril,circuit_id_galvez,circuit_id_hockenheimring,circuit_id_hungaroring,circuit_id_imola,circuit_id_indianapolis,circuit_id_interlagos,circuit_id_istanbul,circuit_id_jacarepagua,circuit_id_jerez,circuit_id_kyalami,circuit_id_magny_cours,circuit_id_marina_bay,circuit_id_monaco,...,nationality_Austrian,nationality_Belgian,nationality_Brazilian,nationality_British,nationality_Canadian,nationality_Dutch,nationality_Finnish,nationality_French,nationality_German,nationality_Italian,nationality_Japanese,nationality_Mexican,nationality_Russian,nationality_Spanish,nationality_Swedish,constructor_arrows,constructor_bar,constructor_benetton,constructor_brabham,constructor_ferrari,constructor_footwork,constructor_force_india,constructor_haas,constructor_jaguar,constructor_jordan,constructor_larrousse,constructor_ligier,constructor_lotus_f1,constructor_mclaren,constructor_mercedes,constructor_minardi,constructor_prost,constructor_red_bull,constructor_renault,constructor_sauber,constructor_team_lotus,constructor_toro_rosso,constructor_toyota,constructor_tyrrell,constructor_williams
0,1983,1,False,False,True,False,False,keke_rosberg,1,15,0,0,0,0,0,0,0.000,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1983,1,False,False,True,False,False,prost,2,6,0,0,0,0,0,0,0.146,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,1983,1,False,False,True,False,False,tambay,3,4,0,0,0,0,0,0,0.232,33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1983,1,False,False,True,False,False,piquet,4,1,0,0,0,0,0,0,0.588,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1983,1,False,False,True,False,False,warwick,5,7,0,0,0,0,0,0,0.680,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Classification and splitting data

In [9]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season <2019]
X_train = train.drop(['driver', 'podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [10]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == 2019]['round'].unique():

        test = df[(df.season == 2019) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop = True)
        prediction_df.sort_values('proba_1', ascending = False, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)
        print(prediction_df)

    model_score = score / df[df.season == 2019]['round'].unique().max()
    return model_score

In [11]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

In [12]:
# Logistic Regression

params={'penalty': ['l1', 'l2'],
        'solver': ['saga', 'liblinear'],
        'C': np.logspace(-3,1,20)}

for penalty in params['penalty']:
    for solver in params['solver']:
        for c in params['C']:
            model_params = (penalty, solver, c)
            model = LogisticRegression(penalty = penalty, solver = solver, C = c, max_iter = 10000)
            model.fit(X_train, y_train)
            
            model_score = score_classification(model)
            
            comparison_dict['model'].append('logistic_regression')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

Streaming output truncated to the last 5000 lines.
16  0.996814  0.003186       0          0
17  0.997544  0.002456       0          0
18  0.997576  0.002424       0          0
     proba_0   proba_1  actual  predicted
0   0.430087  0.569913       0          1
1   0.800102  0.199898       0          0
2   0.837083  0.162917       1          0
3   0.894185  0.105815       0          0
4   0.953088  0.046912       0          0
5   0.978705  0.021295       0          0
6   0.978758  0.021242       0          0
7   0.983700  0.016300       0          0
8   0.987085  0.012915       0          0
9   0.987557  0.012443       0          0
10  0.989201  0.010799       0          0
11  0.990575  0.009425       0          0
12  0.993208  0.006792       0          0
13  0.994415  0.005585       0          0
14  0.994842  0.005158       0          0
15  0.994899  0.005101       0          0
16  0.996466  0.003534       0          0
17  0.997337  0.002663       0          0
18  0.997496  0.002504   

Linear Regression

In [13]:
def score_regression(model):
    score = 0
    for circuit in df[df.season == 2019]['round'].unique():

        test = df[(df.season == 2019) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns = ['results'])
        prediction_df['podium'] = y_test.reset_index(drop = True)
        prediction_df['actual'] = prediction_df.podium.map(lambda x: 1 if x == 1 else 0)
        prediction_df.sort_values('results', ascending = True, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)
        print(prediction_df)
    model_score = score / df[df.season == 2019]['round'].unique().max()
    return model_score

In [14]:
# Linear Regression

params={'fit_intercept': ['True', 'False']}

for fit_intercept in params['fit_intercept']:
    model_params = (fit_intercept)
    model = LinearRegression(fit_intercept = fit_intercept)
    model.fit(X_train, y_train)
            
    model_score = score_regression(model)
            
    comparison_dict['model'].append('linear_regression')
    comparison_dict['params'].append(model_params)
    comparison_dict['score'].append(model_score)

     results  podium  actual  predicted
0  -0.010549       0       0          1
1  -0.009063       0       0          0
2   0.003359       0       0          0
3   0.003980       0       0          0
4   0.013574       0       0          0
5   0.022535       0       0          0
6   0.028131       0       0          0
7   0.031693       0       0          0
8   0.043093       0       0          0
9   0.043878       0       0          0
10  0.050218       0       0          0
11  0.053803       0       0          0
12  0.055012       0       0          0
13  0.056089       0       0          0
14  0.085875       0       0          0
15  0.090471       0       0          0
16  0.121169       0       0          0
17  0.123125       0       0          0
18  0.154913       1       1          0
19  0.179104       0       0          0
     results  podium  actual  predicted
0  -0.008326       0       0          1
1  -0.003105       0       0          0
2  -0.000415       0       0          0


SVM Regressor

In [ ]:
params={'gamma': np.logspace(-4, -1, 10),
        'C': np.logspace(-2, 1, 10),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} 

for gamma in params['gamma']:
    for c in params['C']:
        for kernel in params['kernel']:
            model_params = (gamma, c, kernel)
            model = svm.SVR(gamma = gamma, C = c, kernel = kernel)
            model.fit(X_train, y_train)
            
            model_score = score_regression(model)
            
            comparison_dict['model'].append('svm_regressor')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

Streaming output truncated to the last 5000 lines.
16  0.100547       1       1          0
17  0.100700       0       0          0
18  0.100778       0       0          0
     results  podium  actual  predicted
0   0.098677       0       0          1
1   0.098737       0       0          0
2   0.098911       0       0          0
3   0.098964       0       0          0
4   0.099108       0       0          0
5   0.099194       0       0          0
6   0.099200       0       0          0
7   0.099214       0       0          0
8   0.099316       0       0          0
9   0.099337       0       0          0
10  0.099341       0       0          0
11  0.099425       0       0          0
12  0.099436       0       0          0
13  0.099566       0       0          0
14  0.099904       0       0          0
15  0.100024       0       0          0
16  0.100036       0       0          0
17  0.100461       0       0          0
18  0.100533       1       1          0
     results  podium  actual 

Comparing the Models

In [15]:
pd.DataFrame(comparison_dict).groupby('model')['score'].max()

model
linear_regression      0.000000
logistic_regression    0.571429
Name: score, dtype: float64